In [27]:
import json
import torch
import custom_utils
import numpy as np

import optuna

In [28]:
# importing data
with open('data/X_train.json', 'r') as json_file:
    X_train = json.load(json_file)
   
with open('data/y_train.json', 'r') as json_file:
    y_train = json.load(json_file)
   
with open('data/X_test.json', 'r') as json_file:
    X_test = json.load(json_file) 

with open('data/y_test.json', 'r') as json_file:
    y_test = json.load(json_file)
    
# converting to appropriate format
X_train = torch.Tensor(X_train).float()
y_train = torch.Tensor(y_train).long()

X_test = torch.Tensor(X_test).float()
y_test = torch.Tensor(y_test).long()

# getting shapes
num_features = X_train.shape[1]
num_classes = 2
# edge_dim = edge_attr.shape[1]

In [29]:
from torch.nn import Linear
import torch.nn.functional as F

class MLP(torch.nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(12345)
        self.lin1 = Linear(num_features, 300)
        self.lin2 = Linear(300, 600)
        self.lin3 = Linear(600, num_classes)

    def forward(self, x):
        x = self.lin1(x)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        x = x.sigmoid()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin3(x)
        return x

In [30]:
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
import copy

In [36]:
def objective(trial):
    
    def train(X_train, y_train):
        model.train() 
        optimizer.zero_grad()
        out = model(X_train)
        loss = criterion(out, y_train)
        loss.backward()
        optimizer.step()
        return loss
    
    def validate(X_valid, y_valid):
        model.eval()
        out = model(X_valid)
        loss = criterion(out, y_valid)
        y_pred = out.argmax(dim=1)
        score = f1_score(y_valid, y_pred)
        return loss, score
        
    
    n_folds = 5
    n_epochs = 200
    patience = 10
    avg_score = 0
    skf = StratifiedKFold(n_splits=n_folds)
    for i, (train_idx, val_idx) in enumerate(skf.split(X_train,y_train)):
        
        # Model
        n_layers = trial.suggest_int('n_layers', 1, 4)
        layers = []

        in_features = num_features
        for i in range(n_layers):
            out_features = trial.suggest_int(f'n_units_l{i}', 200, 800)
            layers.append(torch.nn.Linear(in_features, out_features))
            layers.append(torch.nn.ReLU())
            
            # suggest dropout
            # p = trial.suggest_float(f'p{i}', 0, 0.8)
            # layers.append(torch.nn.functional.dropout(p))

            # updating next layer size
            in_features = out_features
            
        layers.append(torch.nn.Linear(in_features, num_classes))
        
        model = torch.nn.Sequential(*layers).to(torch.device('cpu'))
        criterion = torch.nn.CrossEntropyLoss() # need to check input
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
            
        # Selecing fold train and validation
        train_mask = torch.zeros(len(X_train), dtype=torch.bool)
        val_mask = torch.zeros(len(X_train), dtype=torch.bool)
        train_mask[train_idx] = True
        val_mask[val_idx] = True
        
        X_train_fold = X_train[train_mask]
        y_train_fold = y_train[train_mask]
        X_valid_fold = X_train[val_mask]
        y_valid_fold = y_train[val_mask]
        
        # Epoch
        best_valid_loss = float('inf')
        score_at_best = -1
        current_patience = 0
        for epoch in range(n_epochs):
            train_loss = train(X_train_fold, y_train_fold)
            valid_loss, score = validate(X_valid_fold, y_valid_fold)
           
            # Stopping criterium            
            if valid_loss > best_valid_loss:
                current_patience += 1
            else:
                best_valid_loss = valid_loss
                score_at_best = score 
                current_patience = 0
            
            if current_patience == patience:
                break
            
            print(f'Epoch: {epoch}, Train loss: {train_loss:.4f}, Valid loss: {valid_loss:.4f}, Score: {score:.4f}')
        
        avg_score += score_at_best/n_folds
        
    return avg_score

In [37]:
# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2023-11-28 17:28:12,528] A new study created in memory with name: no-name-92712a3d-28ff-411b-874b-a8e35e24c80e


Epoch: 0, Train loss: 0.7099, Valid loss: 0.5187, Score: 0.0000
Epoch: 1, Train loss: 0.5200, Valid loss: 0.4781, Score: 0.0000
Epoch: 2, Train loss: 0.4803, Valid loss: 0.4357, Score: 0.0000
Epoch: 3, Train loss: 0.4377, Valid loss: 0.3766, Score: 0.0000
Epoch: 4, Train loss: 0.3765, Valid loss: 0.3837, Score: 0.0000
Epoch: 5, Train loss: 0.3840, Valid loss: 0.3855, Score: 0.0000
Epoch: 6, Train loss: 0.3859, Valid loss: 0.3665, Score: 0.0000
Epoch: 7, Train loss: 0.3655, Valid loss: 0.3624, Score: 0.0000
Epoch: 8, Train loss: 0.3599, Valid loss: 0.3717, Score: 0.1751
Epoch: 9, Train loss: 0.3686, Valid loss: 0.3674, Score: 0.0518
Epoch: 10, Train loss: 0.3639, Valid loss: 0.3597, Score: 0.0000
Epoch: 11, Train loss: 0.3558, Valid loss: 0.3574, Score: 0.0000
Epoch: 12, Train loss: 0.3531, Valid loss: 0.3585, Score: 0.0000
Epoch: 13, Train loss: 0.3537, Valid loss: 0.3572, Score: 0.0331
Epoch: 14, Train loss: 0.3518, Valid loss: 0.3526, Score: 0.2680
Epoch: 15, Train loss: 0.3464, Vali

[I 2023-11-28 17:32:43,790] Trial 0 finished with value: 0.5280845741424421 and parameters: {'n_layers': 2, 'n_units_l0': 604, 'n_units_l1': 793}. Best is trial 0 with value: 0.5280845741424421.


Epoch: 0, Train loss: 0.6758, Valid loss: 0.5044, Score: 0.0000
Epoch: 1, Train loss: 0.5055, Valid loss: 0.5601, Score: 0.0000
Epoch: 2, Train loss: 0.5608, Valid loss: 0.5212, Score: 0.0000
Epoch: 3, Train loss: 0.5229, Valid loss: 0.4105, Score: 0.0000
Epoch: 4, Train loss: 0.4121, Valid loss: 0.4145, Score: 0.0000
Epoch: 5, Train loss: 0.4172, Valid loss: 0.4060, Score: 0.0000
Epoch: 6, Train loss: 0.4089, Valid loss: 0.3739, Score: 0.0000
Epoch: 7, Train loss: 0.3745, Valid loss: 0.3856, Score: 0.0000
Epoch: 8, Train loss: 0.3858, Valid loss: 0.3822, Score: 0.0000
Epoch: 9, Train loss: 0.3815, Valid loss: 0.3687, Score: 0.0000
Epoch: 10, Train loss: 0.3663, Valid loss: 0.3676, Score: 0.0000
Epoch: 11, Train loss: 0.3642, Valid loss: 0.3704, Score: 0.0000
Epoch: 12, Train loss: 0.3663, Valid loss: 0.3635, Score: 0.0224
Epoch: 13, Train loss: 0.3582, Valid loss: 0.3547, Score: 0.4102
Epoch: 14, Train loss: 0.3485, Valid loss: 0.3511, Score: 0.4862
Epoch: 15, Train loss: 0.3451, Vali

[I 2023-11-28 17:37:06,590] Trial 1 finished with value: 0.5291237255847158 and parameters: {'n_layers': 3, 'n_units_l0': 649, 'n_units_l1': 353, 'n_units_l2': 621}. Best is trial 1 with value: 0.5291237255847158.


Epoch: 0, Train loss: 0.6940, Valid loss: 0.5110, Score: 0.0000
Epoch: 1, Train loss: 0.5114, Valid loss: 0.6062, Score: 0.0000
Epoch: 2, Train loss: 0.6063, Valid loss: 0.5585, Score: 0.0000
Epoch: 3, Train loss: 0.5590, Valid loss: 0.4617, Score: 0.0000
Epoch: 4, Train loss: 0.4634, Valid loss: 0.4885, Score: 0.0000
Epoch: 5, Train loss: 0.4931, Valid loss: 0.3930, Score: 0.0000
Epoch: 6, Train loss: 0.3949, Valid loss: 0.4003, Score: 0.0000
Epoch: 7, Train loss: 0.4014, Valid loss: 0.3758, Score: 0.0000
Epoch: 8, Train loss: 0.3750, Valid loss: 0.3760, Score: 0.0000
Epoch: 9, Train loss: 0.3745, Valid loss: 0.3799, Score: 0.0000
Epoch: 10, Train loss: 0.3782, Valid loss: 0.3654, Score: 0.0000
Epoch: 11, Train loss: 0.3621, Valid loss: 0.3644, Score: 0.1066
Epoch: 12, Train loss: 0.3599, Valid loss: 0.3636, Score: 0.4106
Epoch: 13, Train loss: 0.3587, Valid loss: 0.3534, Score: 0.3065
Epoch: 14, Train loss: 0.3487, Valid loss: 0.3526, Score: 0.1949
Epoch: 15, Train loss: 0.3493, Vali

[I 2023-11-28 17:42:40,657] Trial 2 finished with value: 0.5175603015811081 and parameters: {'n_layers': 4, 'n_units_l0': 211, 'n_units_l1': 456, 'n_units_l2': 519, 'n_units_l3': 704}. Best is trial 1 with value: 0.5291237255847158.


Epoch: 0, Train loss: 0.6982, Valid loss: 0.6439, Score: 0.0000
Epoch: 1, Train loss: 0.6441, Valid loss: 0.6148, Score: 0.0000
Epoch: 2, Train loss: 0.6148, Valid loss: 0.6255, Score: 0.0000
Epoch: 3, Train loss: 0.6255, Valid loss: 0.5963, Score: 0.0000
Epoch: 4, Train loss: 0.5965, Valid loss: 0.5277, Score: 0.0000
Epoch: 5, Train loss: 0.5279, Valid loss: 0.4848, Score: 0.0000
Epoch: 6, Train loss: 0.4857, Valid loss: 0.5047, Score: 0.0000
Epoch: 7, Train loss: 0.5065, Valid loss: 0.4468, Score: 0.0000
Epoch: 8, Train loss: 0.4491, Valid loss: 0.4280, Score: 0.0000
Epoch: 9, Train loss: 0.4309, Valid loss: 0.3968, Score: 0.0000
Epoch: 10, Train loss: 0.3984, Valid loss: 0.3721, Score: 0.0000
Epoch: 11, Train loss: 0.3726, Valid loss: 0.3801, Score: 0.0000
Epoch: 12, Train loss: 0.3800, Valid loss: 0.3685, Score: 0.5380
Epoch: 13, Train loss: 0.3645, Valid loss: 0.3704, Score: 0.5607
Epoch: 14, Train loss: 0.3657, Valid loss: 0.3614, Score: 0.5594
Epoch: 15, Train loss: 0.3555, Vali

[W 2023-11-28 17:46:39,196] Trial 3 failed with parameters: {'n_layers': 4, 'n_units_l0': 561, 'n_units_l1': 350, 'n_units_l2': 586, 'n_units_l3': 722} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/nicolas/Documents/projects/extractive-summarization/env/lib/python3.11/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_52391/379183780.py", line 67, in objective
    train_loss = train(X_train_fold, y_train_fold)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_52391/379183780.py", line 8, in train
    loss.backward()
  File "/home/nicolas/Documents/projects/extractive-summarization/env/lib/python3.11/site-packages/torch/_tensor.py", line 492, in backward
    torch.autograd.backward(
  File "/home/nicolas/Documents/projects/extractive-summarization/env/lib/python3.11/site-packages/torch/autograd/__init__.

KeyboardInterrupt: 